In [2]:
load_dotenv("token.env")

True

In [ ]:
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import time

# Загрузка переменных окружения
load_dotenv(find_dotenv())

# Получаем API ключ
api_key = os.getenv('OPENAI_API_KEY') or os.getenv('JWT_TOKEN')

if not api_key:
    raise ValueError("API ключ не найден. Проверьте .env файл")

# Инициализация клиента
client = OpenAI(
    api_key=api_key,
    base_url="https://corellm.wb.ru/llama3/v1",
)

def generate_search_query(hierarchy):
    try:
        response = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "Ты помощник, который генерирует человеческие поисковые запросы для товаров."
                },
                {
                    "role": "user",
                    "content": f"Сгенерируй наиболее популярный поисковой запрос для товара из каталога маркетплейса с иерархией: {hierarchy}\n\nТолько запрос, без пояснений. Не додумывай и не уточняй запрос, опирайся только на иерахию. Пиши как покупатель с маленькой буквы."
                }
            ],
            model="Meta-Llama-3-70B-Instruct-GPTQ",
            temperature=0.7,
            max_tokens=50
        )
        return response.choices[0].message.content.strip() if response.choices[0].message.content else None
    except Exception as e:
        print(f"Ошибка генерации запроса: {str(e)}")
        return None

def process_excel(input_path, output_path):
    try:
        # Чтение файла
        df = pd.read_excel(input_path)
        
        # Проверка структуры
        if 'hierarchy' not in df.columns:
            raise ValueError("Отсутствует столбец 'hierarchy'")
            
        # Инициализация столбца для результатов
        if 'query' not in df.columns:
            df['query'] = ''
        
        # Обработка данных
        for index, row in df.iterrows():
            try:
                # Безопасное получение и проверка значения
                hierarchy = row['hierarchy']
                
                if pd.isna(hierarchy) or not isinstance(hierarchy, str) or not hierarchy.strip():
                    df.at[index, 'query'] = ''
                    continue
                    
                print(f"Обработка строки {index+1}/{len(df)}: {hierarchy[:50]}...")
                
                query = generate_search_query(hierarchy)
                if query:
                    df.at[index, 'query'] = query
                    print(f"Результат: {query}")
                else:
                    df.at[index, 'query'] = ''
                
                time.sleep(1)  # Задержка между запросами
                
            except Exception as row_error:
                print(f"Ошибка в строке {index+1}: {str(row_error)}")
                continue
        
        # Сохранение результатов
        df.to_excel(output_path, index=False)
        print(f"Готово! Результаты сохранены в {output_path}")
        
        # Показываем первые 5 строк результата
        print("\nПервые 5 строк результата:")
        print(df.head())
    
    except Exception as e:
        print(f"Критическая ошибка: {str(e)}")

if __name__ == "__main__":
    input_file = "input_table.xlsx"
    output_file = "ouyput_table.xlsx"
    
    # Проверка существования файла
    if not os.path.exists(input_file):
        print(f"Ошибка: Файл {input_file} не найден!")
    else:
        process_excel(input_file, output_file)

Обработка строки 1/33: Спорт > Для мужчин > Одежда > Рубашки...
Результат: мужские рубашки для спорта
Обработка строки 2/33: Электроника > Смартфоны и телефоны > Чехлы > Чехлы...
Результат: чехлы для samsung galaxy z flip
Обработка строки 3/33: Электроника > Смартфоны и телефоны > Чехлы > Чехлы...
Результат: чехлы для samsung galaxy z fold
Обработка строки 4/33: Электроника > Смартфоны и телефоны > Чехлы > Чехлы...
Результат: чехлы для philips
Обработка строки 5/33: Книги > Репринтные издания > Репринтные издания 19...
Результат: репринтные издания 1930 года
Обработка строки 6/33: Книги > Репринтные издания > Репринтные издания на...
Результат: репринтные издания на французском языке
Обработка строки 7/33: Электроника > Смартфоны и телефоны > Чехлы > Чехлы...
Результат: чехол для nokia
Обработка строки 8/33: Книги > Репринтные издания > Репринтные издания 19...
Результат: репринтные издания 1920 годов книги
Обработка строки 9/33: Спорт > Зимние виды спорта > Беговые лыжи > Инстру...
Ре

In [13]:
input_file = "output_with_queries.xlsx"
df = pd.read_excel(input_file)
df

,id,search_query,carts,url,hierarchy,query,Был ли пресет такой,Запрос 1 (для перечислений),Запрос 2 (для перечислений),Запрос 3 (для перечислений),Запрос 4 (для перечислений),ФИО,далее из скрипта,Подобранный поисковый запрос,Был ли пресет такой.1,Запрос 1 (для перечислений).1,Запрос 2 (для перечислений).1,ФИО.1
0,129312,menu_redirect_subject_v2_129312 мужские спорти...,0.0,/catalog/sport/dlya-muzhchin/odezhda/rubashki,Спорт > Для мужчин > Одежда > Рубашки,рубашки для мужчин спорта,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0
1,130823,menu_mined_subject_v2_130823,0.0,/catalog/elektronika/smartfony-i-telefony/cheh...,Электроника > Смартфоны и телефоны > Чехлы > Ч...,чехлы для samsung galaxy z flip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,чехлы для телефона samsung Galaxy Z Flip,NaN,0,0,Боровиков А
2,130824,menu_mined_subject_v2_130824,0.0,/catalog/elektronika/smartfony-i-telefony/cheh...,Электроника > Смартфоны и телефоны > Чехлы > Ч...,чехлы для samsung galaxy z fold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,чехлы для телефона samsung Galaxy Z Fold,NaN,0,0,Боровиков А
3,129329,menu_mined_subject_v2_129329,0.0,/catalog/elektronika/smartfony-i-telefony/cheh...,Электроника > Смартфоны и телефоны > Чехлы > Ч...,чехлы для philips,NaN,NaN,NaN,NaN,NaN,NaN,NaN,чехлы для телефона philips,NaN,0,0,Боровиков А
4,128583,menu_mined_subject_v2_128583,0.0,/catalog/knigi/nehudozhestvennaya-literatura/i...,Книги > Репринтные издания > Репринтные издани...,репринтные издания книги 1930 года,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0
5,128586,menu_mined_subject_v2_128586,0.0,/catalog/knigi/nehudozhestvennaya-literatura/i...,Книги > Репринтные издания > Репринтные издани...,репринтные издания на французском языке,NaN,NaN,NaN,NaN,NaN,NaN,NaN,книги по репринтные издания на французском языке,NaN,0,0,Боровиков А
6,129331,menu_mined_subject_v2_129331,0.0,/catalog/elektronika/smartfony-i-telefony/cheh...,Электроника > Смартфоны и телефоны > Чехлы > Ч...,чехлы для nokia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,чехлы для телефона nokia,NaN,0,0,Боровиков А
7,128582,menu_mined_subject_v2_128582,0.0,/catalog/knigi/nehudozhestvennaya-literatura/i...,Книги > Репринтные издания > Репринтные издани...,книги репринтные издания 1920 1929 года,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0
8,60081,NONE,0.0,/catalog/sport/vidy-sporta/zimnie-vidy-sporta/...,Спорт > Зимние виды спорта > Беговые лыжи > Ин...,беговые лыжи инструменты,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0
9,131469,menu_mined_subject_v2_131469,0.0,/catalog/avtotovary/zapchasti-dlya-lodok-i-kat...,Автотовары > Запчасти для лодок и катеров > По...,подшипники для лодки,NaN,NaN,NaN,NaN,NaN,NaN,NaN,подшипники для лодок и катеров,NaN,подшипники для лодок и катеров,вкладыши для лодок и катеров,Боровиков А
